<a href="https://colab.research.google.com/github/intuition-explorer/ML-Algorithms/blob/main/PalmerPenguins_KNN_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold #Kfold is new to me i want to try
from sklearn.preprocessing import StandardScaler, OneHotEncoder #OneHotEncoder might need
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.decomposition import PCA

#NEW to me now
from sklearn.datasets import fetch_openml #can get dataset from openml based on name
from sklearn.pipeline import Pipeline #simplifies the process of applying multisteps: imputation, scaling, and model training, by treating them as a single object
from sklearn.compose import ColumnTransformer #good for heterogenous data: numerical may need scaling and categorical one hot encoding, applies to specific col
from sklearn.impute import SimpleImputer

In [ ]:
data = sns.load_dataset("penguins") #this is a dataframe already
X = data.drop('species', axis=1).copy()
y=data['species'].copy()
X.shape #344 samples, 6 features, y.shape matches features
X.info() #missing features for 2 penguins body and 11 missing sex
#Learnt not to replace data with 0, I'm thinking mean or median but really depends on the type of penguin...

#IMPUTATION-assignment of a value to something based on inference of related data, can try using the pipeline approach

y=y.to_numpy() #Just prudent to do so for later
y=pd.Series(y)
print(y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   island             344 non-null    object 
 1   bill_length_mm     342 non-null    float64
 2   bill_depth_mm      342 non-null    float64
 3   flipper_length_mm  342 non-null    float64
 4   body_mass_g        342 non-null    float64
 5   sex                333 non-null    object 
dtypes: float64(4), object(2)
memory usage: 16.3+ KB
0      Adelie
1      Adelie
2      Adelie
3      Adelie
4      Adelie
        ...  
339    Gentoo
340    Gentoo
341    Gentoo
342    Gentoo
343    Gentoo
Length: 344, dtype: object


In [ ]:
#Need to learn how to identify outliers
stats = X.describe()
print(stats)
stats = stats.T
#COMPARE MIN AND MAX TO Q1 AND Q3
type(stats)
stats.head()

Q1 = stats['25%']
Q3 = stats['75%']
IQR = Q3 - Q1
l_bound = Q1 - 1.5*IQR
u_bound = Q3 + 1.5*IQR

print('\n', l_bound)
print('\n', u_bound)

       bill_length_mm  bill_depth_mm  flipper_length_mm  body_mass_g
count      342.000000     342.000000         342.000000   342.000000
mean        43.921930      17.151170         200.915205  4201.754386
std          5.459584       1.974793          14.061714   801.954536
min         32.100000      13.100000         172.000000  2700.000000
25%         39.225000      15.600000         190.000000  3550.000000
50%         44.450000      17.300000         197.000000  4050.000000
75%         48.500000      18.700000         213.000000  4750.000000
max         59.600000      21.500000         231.000000  6300.000000

 bill_length_mm         25.3125
bill_depth_mm          10.9500
flipper_length_mm     155.5000
body_mass_g          1750.0000
dtype: float64

 bill_length_mm         62.4125
bill_depth_mm          23.3500
flipper_length_mm     247.5000
body_mass_g          6550.0000
dtype: float64


In [ ]:
# Identify outliers in numerical columns
numerical_cols = X.select_dtypes(include=np.number).columns
outlier_condition = (X[numerical_cols] < l_bound) | (X[numerical_cols] > u_bound)

# Select rows with outliers
outliers = X[outlier_condition.any(axis=1)] #checks row wise if any column in that row is true
print(outliers) #outputs the row, so no outliers

Empty DataFrame
Columns: [island, bill_length_mm, bill_depth_mm, flipper_length_mm, body_mass_g, sex]
Index: []


In [ ]:
#Can use this approach when have a huge dataset of features, good way to distinguish when I cant visually see
num_cols = X.select_dtypes(include=[np.number]).columns.tolist() #select only columns with numbers
cat_cols = [c for c in X.columns if c not in num_cols] #pull out other columns
print(num_cols)
print(cat_cols)

['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
['island', 'sex']


In [ ]:
#so tedious to do it this way and have to remove and add columns, use pipeline and Column transformer instead
manXfill = X[['bill_depth_mm', 'body_mass_g', 'bill_length_mm', 'flipper_length_mm']].fillna(X[['bill_depth_mm', 'body_mass_g', 'bill_length_mm', 'flipper_length_mm']].median())

In [ ]:
type(num_cols) #MUST be a list to put into column transformer

list

In [ ]:
preprocessor1 = ColumnTransformer(transformers=[('num_imputer',SimpleImputer(strategy='median'), num_cols), #type of transform in str, then impute, then specified col
                                                ('cat_imputer',SimpleImputer(strategy='most_frequent'), cat_cols)],
                                remainder= 'passthrough') #other cols not affected, passthrough


t_X1 = preprocessor1.fit_transform(X) #outputs numpy, should run on X_train as fit_transform and just .fit on X_val and X_test

In [ ]:
#With Pipeline as well can prevent leakage and does model tuning, CV, etc all in one step good for practice, wont forget training etc
preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('num_imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), num_cols),
    ('cat', Pipeline([
        ('cat_imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
                                                            #unknown cat not present during fit is vec of all 0s, when new test not in training instead of error
                                                            #sprase fals outputs numpy else scipy array which is more memory efficient for larger
    ]), cat_cols)
], remainder='drop') #eliminates any other types of col instead of passthrough

In [ ]:
# Convert string labels into integers
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_int = le.fit_transform(y)       # integer-encoded labels
y_classes = le.classes_           # original class names in order

print(y[:5])       # original labels (e.g. ['Adelie', 'Adelie', ...])
print(y_int[:5])   # encoded labels (e.g. [0, 0, ...])
print(y_classes)   # mapping (e.g. ['Adelie' 'Chinstrap' 'Gentoo'])


0    Adelie
1    Adelie
2    Adelie
3    Adelie
4    Adelie
dtype: object
[0 0 0 0 0]
['Adelie' 'Chinstrap' 'Gentoo']


In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
#This is good for big datasets-X_train,temp,val, tune hyperparameters on validation, but for smaller try StratifiedKFolds

#To use both, ideal: separate test untouched set using above then use remaining for stratification-training and validation, where each rotates
#X_temp, X_test, y_temp, y_test = train_test_split(X, y_int, test_size=0.2, random_state=42, stratify=y_int)
'''

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# use the folds manually to pick one as validation, rest as train
for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_temp,y_temp)):
  X_train = X.iloc[train_idx]
  y_train = y.iloc[train_idx] #if series then .iloc, else numpy none
  X_val = X.iloc[val_idx]
  y_val = y.iloc[val_idx]

  # fit_transform on train
  X_train_processed = preprocessor.fit_transform(X_train)
  # transform on validation
  X_val_processed = preprocessor.transform(X_val)
  X_test_processed = preprocessor.transform(X_test)


  # Can also define model here, fit it to train then cross validate to get score
  model = LogisticRegression(max_iter=1000)
  model.fit(X_train_processed, y_train)
  score = model.score(X_val_processed, y_val)
  scores.append(score)
  print("CV scores:", scores)
  print("Mean:", np.mean(scores))


  print("Train class distribution:", y_train.value_counts().to_dict())
  print("Val   class distribution:", y_val.value_counts().to_dict())
  print()
'''
#Score here outputs a metric depending on model type: classifieraccuracy, regressorcoefficientR, clusterNegativeInertia (sum os square dist to cluster center)

'\n\nskf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)\n# use the folds manually to pick one as validation, rest as train\nfor fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_temp,y_temp)):\n  X_train = X.iloc[train_idx]\n  y_train = y.iloc[train_idx] #if series then .iloc, else numpy none\n  X_val = X.iloc[val_idx]\n  y_val = y.iloc[val_idx]\n\n  # fit_transform on train\n  X_train_processed = preprocessor.fit_transform(X_train)\n  # transform on validation\n  X_val_processed = preprocessor.transform(X_val)\n  X_test_processed = preprocessor.transform(X_test)\n\n  \n  # Can also define model here, fit it to train then cross validate to get score\n  model = LogisticRegression(max_iter=1000)\n  model.fit(X_train_processed, y_train)\n  score = model.score(X_val_processed, y_val)\n  scores.append(score)\n  print("CV scores:", scores)\n  print("Mean:", np.mean(scores))\n  \n\n  print("Train class distribution:", y_train.value_counts().to_dict())\n  print("Val   class

In [ ]:
X_train_processed = preprocessor.fit_transform(X_train)
  # transform on validation
X_test_processed = preprocessor.transform(X_temp)


In [ ]:
'''
#Cleaner way with pipeline
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

for train_idx, val_idx in skf.split(X_temp, y_temp):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    pipe.fit(X_train, y_train)       # pipeline fits preprocessor + model
    score = pipe.score(X_val, y_val) # pipeline transforms + predicts
    print("Fold score:", score)
'''

NameError: name 'LogisticRegression' is not defined

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

KNC = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
KNC.fit(X_train_processed, y_train)
y_pred = KNC.predict(X_test_processed)

# Convert predicted labels to integers using the fitted LabelEncoder
y_pred_int = le.transform(y_pred)

print(classification_report(y_temp, y_pred_int))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        30
           1       0.93      1.00      0.97        14
           2       1.00      1.00      1.00        25

    accuracy                           0.99        69
   macro avg       0.98      0.99      0.98        69
weighted avg       0.99      0.99      0.99        69



In [ ]:
#KNN metric, this is scalar only..
def pairwise_distances(A, B, metric="euclidean"):
    A=np.array(A)
    B=np.array(B)
    if metric == 'euclidean':
      return np.sqrt((A-B)**2)
    if metric == 'manhattan':
      return np.sum(np.abs(A-B))
    else:
      raise ValueError("Unsupported metric")

print(pairwise_distances([1,2],[2,4], 'manhattan'))

3


In [ ]:
#KNN metric vectorized
#A= a_n,d B=b_n,d
def pairwise_distances(A, B, metric="euclidean"):
 if metric == 'euclidean':
  A2 = np.sum(A**2, axis =1, keepdims=True) #squares all vals in the array, then sums them up by the column, then keepsdim: shape n_a,1 instead of n_a
  B2 = np.sum(B**2, axis =1)[None, :] #gets shape n_b, with None,:, then add a column infront to get 1,n_b #good for BROADCASTING
#None adds new axis, here array is a tensor axis=0 row axis=1 column, and : is a slice operator-keeps all the features
#[None,:] goes from n_b to 1, n_b

  AB = A @ B.T #Multiply vectors/arrays instead of scalar approach i tried above and shapes work b/c of steps above
  D2 = A2 + B2 - 2*AB
  D2[D2<0] = 0 #safety due to floating points so get no small negatives,
  return np.sqrt(D2)
 if metric == 'manhattan':
  np.sum(np.abs(A[:, None, :] - B[None, :, :]), axis=2) #A becomes (n_a, 1, d) B: (1, n_b, d)
  #A-B yields (n_a, n_b, d), axis =2 says to sum up only 3rd axis here
 else:
  raise ValueError("Unsupported metric")


In [ ]:
#KNN- majority vote
# (distance_weighted = False): didnt include if, distance weighted false is take avg or majority, if true closer values matter more

class KNN:
  def __init__(self, k=1, metric = 'euclidean'):
    self.k = k #self is the instance of the object, allows object to store its own data & parameters
    self.metric = metric
   # self.distance_weighted = distance_weighted
    self.X_ = None #place holders for data later, when define fit store them
    self.y_ = None
    self.classes_ = None

  def fit(self, X, y): #later we will do model.fit or I guess KNN.fit
    self.X_ = np.array(X)
    self.y_ = np.array(y_int)
    self.classes_ = np.unique(self.y_)
    return self #stores everything in its memory above so can do method chaining
#So far just memorized training set, stores classes and hyperparameters

  def predict(self, X):
    X = np.array(X) #new test points you wan to compare, makes np array from data frame
    D = pairwise_distances(X, self.X_, metric=self.metric) #[n_test, n_train] computes distance between all test and training points

#D.shape thus has rows=test points & columns=training points (from dimensions of X_test, X_train)
#if 2 by 2 test and 3 x 2 training, then D is 2 x 3 matrix full of Euclidean distances

    #Dont need for vectorized :y_pred = []

#D.shape[0] then is number of rows=test points (distances from diff targets on columns)

    for i in range(D.shape[0]): #loop over each test point, e.g. i[0] is test (1,2) i[1] test (2,3)
      #neighbors_idx = np.argsort(D[i])[:k] #this gets indices of sorted distances arranged from smallest to highest, the [:k] then takes first k indices

      #Vectorize above, faster and does all rows at once instead of one at a time
      neighbors_idx = np.argsort(D, axis=1)[:, :self.k]  # shape [n_test, k]
#axis=1 is row for a test point now across all training points, this outputs an array of indicies of training points (to test point) in order
# the : is same as 0:end, so takes all rows then :self.k takes columns from 0 to k-1

      neighbor_label = self.y_[neighbors_idx]#from the indices of these points(training) get the corresponding y_label
      #This is now shape [n_test, k]

     # pred = np.bincount(neighbor_label).argmax() #count labels if [0,1] then yields [1,1] (1 for each 0 and 1) then arg max, picks first max: here 0
      #if bincount of label [1,1] then yields 0 0s and 2 1s=[0,2] then picks max 2: so label is 1

      #vectorize above now
      y_pred = np.array([np.bincount(row, minlength=len(self.classes_)).argmax() for row in neighbor_label])
#for row in n_label: loop over test point's neighbours, each row is k,
#min length counts number of classes (works for more than 2), here just 2, so bincount here counts # of classes occurring in a row
    return y_pred

In [ ]:
from sklearn.preprocessing import LabelEncoder
#THIS WAS A FIX PUTTING LABEL ENCODER IN THE LOOP

class KNN:
    def __init__(self, k=1, metric="euclidean"):
        self.k = k
        self.metric = metric
        self.X_ = None
        self.y_ = None
        self.classes_ = None
        self.le = None  # will hold the encoder

    def fit(self, X, y):
        self.X_ = np.array(X)
        self.le = LabelEncoder()
        self.y_ = self.le.fit_transform(y)  # now guaranteed 0..n-1
        self.classes_ = self.le.classes_
        return self

    def predict(self, X):
        X = np.array(X)
        D = pairwise_distances(X, self.X_, metric=self.metric)
        neighbors_idx = np.argsort(D, axis=1)[:, :self.k]
        neighbor_label = self.y_[neighbors_idx]   # now ints 0..n-1
        y_pred_int = np.array([
            np.bincount(row, minlength=len(self.classes_)).argmax()
            for row in neighbor_label
        ])
        return self.le.inverse_transform(y_pred_int)


In [ ]:
clf = KNN()
clf.fit(X_train_processed, y_train)               # y_train = strings
y_test_pred = clf.predict(X_test_processed)       # y_test_pred = strings
print(confusion_matrix(y_temp, y_test_pred))
print(classification_report(y_temp, y_test_pred))


#IT WORKS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# I HAD ISSUE WITH Y BEING STRINGS VS INTS

[[29  1  0]
 [ 0 14  0]
 [ 0  0 25]]
              precision    recall  f1-score   support

      Adelie       1.00      0.97      0.98        30
   Chinstrap       0.93      1.00      0.97        14
      Gentoo       1.00      1.00      1.00        25

    accuracy                           0.99        69
   macro avg       0.98      0.99      0.98        69
weighted avg       0.99      0.99      0.99        69



In [ ]:
# Initialize classifier
clf = KNN()
# Fit on **processed training data**
clf.fit(X_train_processed, y_train)

#predict on
y_test_pred = clf.predict(X_test_processed)

print(classification_report(y_test, y_test_pred))
print(np.unique(y_int))

print(confusion_matrix(y_test, y_test_pred))
#y_test_pred_int = le.transform(y_test_pred)

#print(classification_report(y_test, y_test_pred_int))
#okay works
'''
           0       0.79      1.00      0.88        30
           1       0.19      0.43      0.27        14
           2       0.00      0.00      0.00        25

    accuracy                           0.52        69
   macro avg       0.33      0.48      0.38        69
weighted avg       0.38      0.52      0.44        69
'''

ValueError: Mix of label input types (string and number)

In [ ]:
KNC.fit(X_train_processed, y_train)
y_pred = KNC.predict(X_test_processed)

# Convert predicted labels to integers using the fitted LabelEncoder
y_pred_int = le.transform(y_pred)

print(classification_report(y_test, y_pred_int))
#Same as chat GPT

              precision    recall  f1-score   support

           0       1.00      0.97      0.98        30
           1       0.93      1.00      0.97        14
           2       1.00      1.00      1.00        25

    accuracy                           0.99        69
   macro avg       0.98      0.99      0.98        69
weighted avg       0.99      0.99      0.99        69



In [ ]:
#STUDY THEIR PERFECT OUTPUT:

In [ ]:
# 0) Repro + Imports
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.decomposition import PCA

rng = np.random.RandomState(42)

# 1) Load data (Palmer Penguins)
# Option A: seaborn.load_dataset('penguins') if you prefer.
import seaborn as sns
df = sns.load_dataset("penguins").dropna(subset=["species"])  # keep rows with species label

# 2) Define target + features
y = df["species"].to_numpy()
X = df.drop(columns=["species"])

# Identify column types
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

# 3) Preprocess: impute + encode + scale
from sklearn.impute import SimpleImputer
pre = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("impute", SimpleImputer(strategy="median")),
            ("scale", StandardScaler())
        ]), num_cols),
        ("cat", Pipeline([
            ("impute", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
        ]), cat_cols)
    ],
    remainder="drop"
)

# 4) Train/val/test split (stratified)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

X_trainval_prep = pre.fit_transform(X_trainval)
X_test_prep = pre.transform(X_test)

# 5) From-scratch k-NN (classifier)
def pairwise_distances(A, B, metric="euclidean"):
    # A: [n_a, d], B: [n_b, d]
    if metric == "euclidean":
        # (a - b)^2 = a^2 + b^2 - 2ab
        A2 = np.sum(A**2, axis=1, keepdims=True)       # [n_a, 1]
        B2 = np.sum(B**2, axis=1)[None, :]             # [1, n_b]
        AB = A @ B.T                                   # [n_a, n_b]
        D2 = A2 + B2 - 2*AB
        D2[D2 < 0] = 0
        return np.sqrt(D2)
    elif metric == "manhattan":
        # |a - b| summed over dims
        return np.sum(np.abs(A[:, None, :] - B[None, :, :]), axis=2)
    else:
        raise ValueError("Unsupported metric")

class KNNClassifier:
    def __init__(self, k=5, metric="euclidean", distance_weighted=False):
        self.k = k
        self.metric = metric
        self.distance_weighted = distance_weighted
        self.X_ = None
        self.y_ = None
        self.classes_ = None

    def fit(self, X, y):
        self.X_ = np.array(X, dtype=float)
        self.y_ = np.array(y)
        self.classes_ = np.unique(self.y_)
        return self

    def predict(self, X):
        X = np.array(X, dtype=float)
        D = pairwise_distances(X, self.X_, metric=self.metric)  # [n_test, n_train]
        # argsort to find k nearest for each row
        nn_idx = np.argpartition(D, kth=self.k-1, axis=1)[:, :self.k]  # [n_test, k]
        preds = []
        for i in range(X.shape[0]):
            idx = nn_idx[i]
            labels = self.y_[idx]
            if self.distance_weighted:
                # inverse distance weights (add small epsilon)
                eps = 1e-8
                weights = 1.0 / (D[i, idx] + eps)
                # vote per class
                votes = {c: 0.0 for c in self.classes_}
                for lab, w in zip(labels, weights):
                    votes[lab] += w
                pred = max(votes.items(), key=lambda x: x[1])[0]
            else:
                # majority vote with deterministic tie-break
                values, counts = np.unique(labels, return_counts=True)
                max_count = np.max(counts)
                tied = values[counts == max_count]
                # tie-break by closest neighbor among tied classes
                if len(tied) > 1:
                    # pick class of the single closest neighbor among those tied
                    closest_idx = idx[np.argmin(D[i, idx])]
                    pred = self.y_[closest_idx]
                    if pred not in tied:
                        pred = np.random.choice(tied)  # fallback
                else:
                    pred = tied[0]
            preds.append(pred)
        return np.array(preds)

# 6) Hyperparameter search for k
def tune_k(X, y, ks=(1,3,5,7,9,11), metric="euclidean", distance_weighted=False, folds=5):
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    results = []
    for k in ks:
        fold_scores = []
        for tr_idx, va_idx in skf.split(X, y):
            Xtr, Xva = X[tr_idx], X[va_idx]
            ytr, yva = y[tr_idx], y[va_idx]
            clf = KNNClassifier(k=k, metric=metric, distance_weighted=distance_weighted).fit(Xtr, ytr)
            yhat = clf.predict(Xva)
            fold_scores.append(accuracy_score(yva, yhat))
        results.append((k, np.mean(fold_scores)))
    best_k, best_acc = max(results, key=lambda t: t[1])
    return best_k, results

best_k, cv_table = tune_k(X_trainval_prep, y_trainval, ks=range(1,21,2), metric="euclidean", distance_weighted=True)
print("Best k:", best_k)

# 7) Train final model on all train+val, evaluate on held-out test
clf = KNNClassifier(k=best_k, metric="euclidean", distance_weighted=True).fit(X_trainval_prep, y_trainval)
y_pred = clf.predict(X_test_prep)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# 8) (Optional) Visualize decision regions via PCA→2D
pca = PCA(n_components=2, random_state=42)
Z_train = pca.fit_transform(X_trainval_prep)
Z_test = pca.transform(X_test_prep)

clf2d = KNNClassifier(k=best_k, metric="euclidean", distance_weighted=True).fit(Z_train, y_trainval)
y_pred_2d = clf2d.predict(Z_test)
print("PCA-2D kNN Test Accuracy:", accuracy_score(y_test, y_pred_2d))


Best k: 1
Test Accuracy: 0.9855072463768116
              precision    recall  f1-score   support

      Adelie       1.00      0.97      0.98        30
   Chinstrap       0.93      1.00      0.97        14
      Gentoo       1.00      1.00      1.00        25

    accuracy                           0.99        69
   macro avg       0.98      0.99      0.98        69
weighted avg       0.99      0.99      0.99        69

PCA-2D kNN Test Accuracy: 0.8695652173913043


In [ ]:
#Unrelated -Logistic Regression checking something
# Logistic regression: loss, gradient, Hessian, Fisher metric, and a natural-gradient step
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def neg_log_lik(theta, X, y):
    z = X @ theta
    p = sigmoid(z)
    # stabilize
    eps = 1e-12
    return -np.sum(y*np.log(p+eps) + (1-y)*np.log(1-p+eps))

def grad(theta, X, y):
    p = sigmoid(X @ theta)
    return X.T @ (p - y)

def hessian(theta, X):
    p = sigmoid(X @ theta)
    W = p*(1-p)                  # shape (n,)
    return X.T * W @ X           # X.T @ diag(W) @ X, vectorized

def fisher(theta, X):
    # For logistic regression, expected Hessian = Fisher = X^T diag(p*(1-p)) X
    return hessian(theta, X)

def natural_grad_step(theta, X, y, lr=1.0, damping=1e-4):
    g  = grad(theta, X, y)
    F  = fisher(theta, X)
    # damped solve (Tikhonov) for numerical stability
    step = np.linalg.solve(F + damping*np.eye(F.shape[0]), g)
    return theta - lr * step

# Diagnostics: SVD & principal directions of curvature
def curvature_decomp(theta, X):
    # SVD of data operator
    U, S, Vt = np.linalg.svd(X, full_matrices=False)
    # Hessian/Fisher at theta
    H = hessian(theta, X)
    evals, evecs = np.linalg.eigh(H)  # symmetric PSD
    return (U, S, Vt.T), (evals, evecs)

# Example usage (toy):
# X: (n_samples x d_features), y in {0,1}
# theta0 = np.zeros(d_features)
# for _ in range(10):
#     theta0 = natural_grad_step(theta0, X, y, lr=1.0, damping=1e-3)
# (U,S,V), (lam, Q) = curvature_decomp(theta0, X)
